# Generate HAI dataframe

In [2]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = '/Users/kenlocey/Desktop/Rush/CMS_HospitalArchives/'

In [3]:
df = pd.read_pickle(main_dir + 'HRRP/CombinedFiles_HRRP/' + 'Facility.pkl')
print('HRRP df:', df.shape)

df.head()

HRRP df: (745170, 14)


,End Date,Excess Readmission Ratio,Expected Readmission Rate,Facility ID,Facility Name,Footnote,Measure Name,Number of Discharges,Number of Readmissions,Predicted Readmission Rate,Start Date,State,file_month,file_year
0,06/30/2021,1.0301,4.4481,010001,SOUTHEAST HEALTH MEDICAL CENTER,NaN,READM-30-HIP-KNEE-HRRP,NaN,Too Few to Report,4.5819,07/01/2018,AL,01,2023
1,06/30/2021,0.9903,19.3765,010001,SOUTHEAST HEALTH MEDICAL CENTER,NaN,READM-30-COPD-HRRP,202.0,38,19.1885,07/01/2018,AL,01,2023
2,06/30/2021,0.9958,14.5498,010001,SOUTHEAST HEALTH MEDICAL CENTER,NaN,READM-30-AMI-HRRP,319.0,46,14.4888,07/01/2018,AL,01,2023
3,06/30/2021,0.9568,17.7701,010001,SOUTHEAST HEALTH MEDICAL CENTER,NaN,READM-30-PN-HRRP,437.0,71,17.0029,07/01/2018,AL,01,2023
4,06/30/2021,1.0551,21.8333,010001,SOUTHEAST HEALTH MEDICAL CENTER,NaN,READM-30-HF-HRRP,757.0,178,23.0374,07/01/2018,AL,01,2023


In [4]:
print(df['Measure Name'].unique())

['READM-30-HIP-KNEE-HRRP' 'READM-30-COPD-HRRP' 'READM-30-AMI-HRRP'
 'READM-30-PN-HRRP' 'READM-30-HF-HRRP' 'READM-30-CABG-HRRP'
 'READM_30_AMI_HRRP' 'READM_30_CABG_HRRP' 'READM_30_COPD_HRRP'
 'READM_30_HF_HRRP' 'READM_30_HIP_KNEE_HRRP' 'READM_30_PN_HRRP'
 'Acute Myocardial Infarction (AMI) 30-Day Readmissions'
 'Heart Failure (HF) 30-Day Readmissions'
 'Pneumonia (PN) 30-Day Readmissions']


In [5]:
df = df.filter(items=['Facility ID',
                      'file_month',
                      'file_year',
                      'Start Date',
                      'End Date', 
                      'Measure Name',
                      'Excess Readmission Ratio', 
                      'Expected Readmission Rate', 
                      'Number of Discharges',
                      'Number of Readmissions',
                      'Predicted Readmission Rate',
                      'Footnote',
                     ], axis=1)

print(df.shape)

(745170, 12)


In [6]:
d = {
     'READM-30-HIP-KNEE-HRRP': 'READM-30-HIP-KNEE', 
     'READM-30-COPD-HRRP': 'READM-30-COPD',
     'READM-30-AMI-HRRP': 'READM-30-AMI',
     'READM-30-PN-HRRP': 'READM-30-PN',
     'READM-30-HF-HRRP': 'READM-30-HF',
     'READM-30-CABG-HRRP': 'READM-30-CABG',
     'READM_30_AMI_HRRP': 'READM-30-AMI',
     'READM_30_CABG_HRRP': 'READM-30-CABG',
     'READM_30_COPD_HRRP': 'READM-30-COPD',
     'READM_30_HF_HRRP': 'READM-30-HF',
     'READM_30_HIP_KNEE_HRRP': 'READM-30-HIP-KNEE',
     'READM_30_PN_HRRP': 'READM-30-PN',
     'Acute Myocardial Infarction (AMI) 30-Day Readmissions': 'READM-30-AMI',
     'Heart Failure (HF) 30-Day Readmissions': 'READM-30-HF', 
     'Pneumonia (PN) 30-Day Readmissions': 'READM-30-PN',
     }

df['Measure Name'].replace(to_replace=d, inplace=True)


In [8]:
Measures = sorted(df['Measure Name'].unique())
main_df = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

for i, mi in enumerate(Measures):
    
    tdf = df[df['Measure Name'] == mi]
    measures = sorted(tdf['Measure Name'].unique())
    
    df2 = pd.DataFrame(columns=['Facility ID', 'file_month', 'file_year'])

    for j, m in enumerate(measures):
    
        tdf2 = tdf[tdf['Measure Name'] == m]
        for n in list(tdf2):
            if n == 'Measure Name' or n in ['Facility ID', 'file_month', 'file_year']:
                continue

            else:
                tdf2[n] = pd.to_numeric(tdf2[n], errors='coerce')
                tdf2.rename(columns={n: m + ' (' + n + ')'}, inplace=True)
        
        tdf2.drop(labels=['Measure Name'], axis=1, inplace=True)
        
        df2 = df2.merge(tdf2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')
    
    main_df = main_df.merge(df2, on=['Facility ID', 'file_month', 
                                          'file_year'], how='outer')



In [9]:
del df2
df = main_df.copy(deep=True)
del main_df

print(df.shape)
df = df.loc[:, ~df.T.duplicated(keep='first')]
df.dropna(how='all', axis=1, inplace=True)
print(df.shape)
df.head()


(141676, 51)
(141676, 39)


,Facility ID,file_month,file_year,READM-30-AMI (Excess Readmission Ratio),READM-30-AMI (Expected Readmission Rate),READM-30-AMI (Number of Discharges),READM-30-AMI (Number of Readmissions),READM-30-AMI (Predicted Readmission Rate),READM-30-AMI (Footnote),READM-30-CABG (Excess Readmission Ratio),READM-30-CABG (Expected Readmission Rate),READM-30-CABG (Number of Discharges),READM-30-CABG (Number of Readmissions),READM-30-CABG (Predicted Readmission Rate),READM-30-CABG (Footnote),READM-30-COPD (Excess Readmission Ratio),READM-30-COPD (Expected Readmission Rate),READM-30-COPD (Number of Discharges),READM-30-COPD (Number of Readmissions),READM-30-COPD (Predicted Readmission Rate),READM-30-COPD (Footnote),READM-30-HF (Excess Readmission Ratio),READM-30-HF (Expected Readmission Rate),READM-30-HF (Number of Discharges),READM-30-HF (Number of Readmissions),READM-30-HF (Predicted Readmission Rate),READM-30-HF (Footnote),READM-30-HIP-KNEE (Excess Readmission Ratio),READM-30-HIP-KNEE (Expected Readmission Rate),READM-30-HIP-KNEE (Number of Discharges),READM-30-HIP-KNEE (Number of Readmissions),READM-30-HIP-KNEE (Predicted Readmission Rate),READM-30-HIP-KNEE (Footnote),READM-30-PN (Excess Readmission Ratio),READM-30-PN (Expected Readmission Rate),READM-30-PN (Number of Discharges),READM-30-PN (Number of Readmissions),READM-30-PN (Predicted Readmission Rate),READM-30-PN (Footnote)
0,010001,01,2023,0.9958,14.5498,319.0,46.0,14.4888,NaN,0.9836,11.3202,165.0,18.0,11.1341,NaN,0.9903,19.3765,202.0,38.0,19.1885,NaN,1.0551,21.8333,757.0,178.0,23.0374,NaN,1.0301,4.4481,NaN,NaN,4.5819,NaN,0.9568,17.7701,437.0,71.0,17.0029,NaN
1,010005,01,2023,0.9603,16.9033,NaN,NaN,16.2327,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.9034,18.2089,235.0,31.0,16.4506,NaN,1.0057,20.0336,157.0,32.0,20.1475,NaN,0.9816,4.1886,NaN,NaN,4.1113,NaN,0.9777,16.0472,361.0,55.0,15.6895,NaN
2,010006,01,2023,1.1127,12.5406,312.0,49.0,13.9536,NaN,1.3316,11.9544,109.0,24.0,15.9187,NaN,0.9027,18.2945,235.0,31.0,16.5144,NaN,0.9440,20.7534,550.0,104.0,19.5920,NaN,1.1885,4.1172,246.0,15.0,4.8932,NaN,1.1525,16.5034,538.0,113.0,19.0197,NaN
3,010007,01,2023,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,0.9979,18.3236,72.0,13.0,18.2845,NaN,1.1129,19.3933,51.0,18.0,21.5828,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.9628,13.2730,NaN,NaN,12.7792,NaN
4,010008,01,2023,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0


In [10]:
for i, n in enumerate(list(df)):
    if n in ['Facility ID', 'file_month', 'file_year']:
        continue
            
    df.rename(columns={n: 'HRRP: ' + n}, inplace=True)
    
print(df.shape)
df.head()

(141676, 39)


,Facility ID,file_month,file_year,HRRP: READM-30-AMI (Excess Readmission Ratio),HRRP: READM-30-AMI (Expected Readmission Rate),HRRP: READM-30-AMI (Number of Discharges),HRRP: READM-30-AMI (Number of Readmissions),HRRP: READM-30-AMI (Predicted Readmission Rate),HRRP: READM-30-AMI (Footnote),HRRP: READM-30-CABG (Excess Readmission Ratio),HRRP: READM-30-CABG (Expected Readmission Rate),HRRP: READM-30-CABG (Number of Discharges),HRRP: READM-30-CABG (Number of Readmissions),HRRP: READM-30-CABG (Predicted Readmission Rate),HRRP: READM-30-CABG (Footnote),HRRP: READM-30-COPD (Excess Readmission Ratio),HRRP: READM-30-COPD (Expected Readmission Rate),HRRP: READM-30-COPD (Number of Discharges),HRRP: READM-30-COPD (Number of Readmissions),HRRP: READM-30-COPD (Predicted Readmission Rate),HRRP: READM-30-COPD (Footnote),HRRP: READM-30-HF (Excess Readmission Ratio),HRRP: READM-30-HF (Expected Readmission Rate),HRRP: READM-30-HF (Number of Discharges),HRRP: READM-30-HF (Number of Readmissions),HRRP: READM-30-HF (Predicted Readmission Rate),HRRP: READM-30-HF (Footnote),HRRP: READM-30-HIP-KNEE (Excess Readmission Ratio),HRRP: READM-30-HIP-KNEE (Expected Readmission Rate),HRRP: READM-30-HIP-KNEE (Number of Discharges),HRRP: READM-30-HIP-KNEE (Number of Readmissions),HRRP: READM-30-HIP-KNEE (Predicted Readmission Rate),HRRP: READM-30-HIP-KNEE (Footnote),HRRP: READM-30-PN (Excess Readmission Ratio),HRRP: READM-30-PN (Expected Readmission Rate),HRRP: READM-30-PN (Number of Discharges),HRRP: READM-30-PN (Number of Readmissions),HRRP: READM-30-PN (Predicted Readmission Rate),HRRP: READM-30-PN (Footnote)
0,010001,01,2023,0.9958,14.5498,319.0,46.0,14.4888,NaN,0.9836,11.3202,165.0,18.0,11.1341,NaN,0.9903,19.3765,202.0,38.0,19.1885,NaN,1.0551,21.8333,757.0,178.0,23.0374,NaN,1.0301,4.4481,NaN,NaN,4.5819,NaN,0.9568,17.7701,437.0,71.0,17.0029,NaN
1,010005,01,2023,0.9603,16.9033,NaN,NaN,16.2327,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.9034,18.2089,235.0,31.0,16.4506,NaN,1.0057,20.0336,157.0,32.0,20.1475,NaN,0.9816,4.1886,NaN,NaN,4.1113,NaN,0.9777,16.0472,361.0,55.0,15.6895,NaN
2,010006,01,2023,1.1127,12.5406,312.0,49.0,13.9536,NaN,1.3316,11.9544,109.0,24.0,15.9187,NaN,0.9027,18.2945,235.0,31.0,16.5144,NaN,0.9440,20.7534,550.0,104.0,19.5920,NaN,1.1885,4.1172,246.0,15.0,4.8932,NaN,1.1525,16.5034,538.0,113.0,19.0197,NaN
3,010007,01,2023,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,0.9979,18.3236,72.0,13.0,18.2845,NaN,1.1129,19.3933,51.0,18.0,21.5828,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.9628,13.2730,NaN,NaN,12.7792,NaN
4,010008,01,2023,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0


In [11]:
df.to_pickle('dataframe_data/hrrp_df.pkl', protocol=5)